<a href="https://colab.research.google.com/github/tomonari-masada/courses/blob/master/Hiroshima_Univ_Topic_Modeling_HandsOn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第1回AI・データイノベーションセミナー 2021年3月11日（木曜日）

## トピックモデリングの実践

* トピックモデリングを、NMF(nonnegative matrix factorization)とLDA(latent Dirichlet allocation)とで実践してみる。
* いずれもscikit-learnの実装を使う。
* 各トピックの上位単語はワードクラウドで可視化する。

* 参考資料
 * https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html



---



## 01 データセットの準備
* NeurIPSで発表された1,740本の論文の本文を使う。

### データをダウンロードしリスト化する関数を定義

* PATHで指定した場所に文書ファイルが配置される。

In [1]:
import io
import os.path
import re
import tarfile
import smart_open


PATH = './' # ここは適当に設定


def extract_documents(url='https://cs.nyu.edu/~roweis/data/nips12raw_str602.tgz'):

  fname = os.path.join(PATH, url.split('/')[-1])

  if not os.path.isfile(fname):
    with smart_open.open(url, "rb") as fin:
      with smart_open.open(fname, 'wb') as fout:
        while True:
          buf = fin.read(io.DEFAULT_BUFFER_SIZE)
          if not buf:
            break
          fout.write(buf)

  with tarfile.open(fname, mode='r:gz') as tar:
  # Ignore directory entries, as well as files like README, etc.
    files = [
             m for m in tar.getmembers()
             if m.isfile() and re.search(r'nipstxt/nips\d+/\d+\.txt', m.name)
             ]
    for member in sorted(files, key=lambda x: x.name):
      member_bytes = tar.extractfile(member).read()
      yield member_bytes.decode('utf-8', errors='replace')

* 実際にデータを取得しリスト化する

In [2]:
docs = list(extract_documents())

* 文書数、具体的な文書の内容などを確認

In [3]:
print(len(docs))

1740


In [4]:
print(docs[0][:1000])

1 
CONNECTIVITY VERSUS ENTROPY 
Yaser S. Abu-Mostafa 
California Institute of Technology 
Pasadena, CA 91125 
ABSTRACT 
How does the connectivity of a neural network (number of synapses per 
neuron) relate to the complexity of the problems it can handle (measured by 
the entropy)? Switching theory would suggest no relation at all, since all Boolean 
functions can be implemented using a circuit with very low connectivity (e.g., 
using two-input NAND gates). However, for a network that learns a problem 
from examples using a local learning rule, we prove that the entropy of the 
problem becomes a lower bound for the connectivity of the network. 
INTRODUCTION 
The most distinguishing feature of neural networks is their ability to spon- 
taneously learn the desired function from 'training' samples, i.e., their ability 
to program themselves. Clearly, a given neural network cannot just learn any 
function, there must be some restrictions on which networks can learn which 
functions. One obv

### spaCyを使ってtokenizeする
* spaCyについては https://spacy.io/ を参照

* 前処理の高速化のためtaggerなどは無効にしておく

In [5]:
import spacy

nlp = spacy.load('en', disable=["tagger", "parser", "ner"])

* テキストを小文字にしてからtokenizeする関数の定義

In [6]:
def spacy_lemmatize_text(nlp, text):
  text = nlp(text.lower())
  doc = [word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text]
  return [word for word in doc if len(word) > 1] # 長さ1の単語は削除

* tokenizationの実行

In [ ]:
from tqdm import tqdm

new_docs = list()
for doc in tqdm(docs):
  new_docs.append(spacy_lemmatize_text(nlp, doc))

 51%|█████     | 882/1740 [00:30<00:31, 27.66it/s]

* tokenizationの結果を確認

In [ ]:
print(new_docs[0])

* 各文書を長い文字列で表しなおす（後でCountVectorizerを使うため）

In [ ]:
corpus = [' '.join(doc) for doc in new_docs]

In [ ]:
corpus[0]

## 02 データ行列の作成
* NMFの場合、TF-IDFで単語の重みを求めて各文書をベクトル化する。
* LDAの場合、単に単語の出現頻度を重みとして各文書をベクトル化する。

### sklearnのCountVectorizerで疎行列化する

* 全文書の半分より多い文書に現れる単語は、高頻度語とみなして削除する。
* 10件未満の文書にしか現れない単語は、低頻度語とみなして削除する。

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_df=0.5, min_df=10, stop_words='english')
X = vectorizer.fit_transform(corpus)

In [ ]:
print(X[0])

In [ ]:
print(vectorizer.get_feature_names())

In [ ]:
print(len(vectorizer.get_feature_names()))

* 文書数と語彙サイズを変数にセット

In [ ]:
X.shape

In [ ]:
n_samples, n_features = X.shape

### TF-IDFで各文書における単語の重みを計算する

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer()
Xtfidf = tfidf.fit_transform(X)

In [ ]:
print(Xtfidf[0])

In [ ]:
Xtfidf.shape

* 抽出するトピックの個数は、今回は20個とする。

In [ ]:
n_components = 20

## 03 NMFでトピック抽出
* まず、TF-IDFのデータ行列を使って　NMFによってトピック抽出を試みる。
 * NMFのパラメータ群は下記サンプルコードを参考にした。
 * https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-plot-topics-extraction-with-nmf-lda-py

### NMFとLDAのインポート

In [ ]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

### NMFによるトピック抽出の実行

In [ ]:
from time import time

print((f"Fitting the NMF model (generalized Kullback-Leibler "
  f"divergence) with tf-idf features, n_samples={n_samples} "
  f"and n_features={n_features}"))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1, l1_ratio=.5,
          verbose=1)
nmf.fit(Xtfidf)
print(f"done in {time() - t0:0.3f}s.")

* NMFにおける各コンポーネントは、それぞれのトピックにおける単語の重要度を表すベクトルとして表現されている。

In [ ]:
nmf.components_

### トピックの重要語を取り出す関数の定義

In [ ]:
def get_top_words(model, feature_names, n_top_words=30):
  top_features = list()
  weights = list()
  for topic_idx, topic in enumerate(model.components_):
    top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
    top_features.append([feature_names[i] for i in top_features_ind])
    weights.append(topic[top_features_ind])
  return top_features, weights

### NMFの各コンポーネントから重要語を取り出す

In [ ]:
top_words, weights = get_top_words(nmf, vectorizer.get_feature_names())

In [ ]:
print(top_words[0])

In [ ]:
topic_words = [dict(zip(top_words[i], weights[i])) for i in range(n_components)]

In [ ]:
topic_words[0]

### 重要語をワードクラウドで可視化

In [ ]:
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS

* ワードクラウドから除去するストップワードを確認する。

In [ ]:
print(STOPWORDS)

* ワードクラウドを描画

In [ ]:
cloud = WordCloud(stopwords=STOPWORDS,
                  background_color='white',
                  width=2500,
                  height=1800,
                  max_words=100,
                  colormap='tab10'
                  )

In [ ]:
fig, axes = plt.subplots(5, 4, figsize=(16, 25), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
  fig.add_subplot(ax)
  cloud.generate_from_frequencies(topic_words[i], max_font_size=300)
  plt.gca().imshow(cloud)
  plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
  plt.gca().axis('off')

plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()

## 04 scikit-learnのLDAでトピック抽出

### LDAによるトピック抽出の実行

In [ ]:
lda = LatentDirichletAllocation(n_components=n_components, 
                                max_iter=20,
                                topic_word_prior=0.01,
                                learning_method='online',
                                learning_offset=50,
                                batch_size=200,
                                mean_change_tol=1e-4,
                                random_state=1,
                                evaluate_every=1,
                                verbose=1)

In [ ]:
print((f"Fitting LDA models with tf features, "
  f"n_samples={n_samples} and n_features={n_features}"))
t0 = time()
lda.fit(X)
print(f"done in {time() - t0:0.3f}s.")

### LDAの各トピックから高確率語を取り出す

In [ ]:
top_words, weights = get_top_words(lda, vectorizer.get_feature_names())

In [ ]:
print(top_words[0])

In [ ]:
topic_words = [dict(zip(top_words[i], weights[i])) for i in range(n_components)]

### 高確率語をワードクラウドで可視化

In [ ]:
cloud = WordCloud(stopwords=STOPWORDS,
                  background_color='white',
                  width=2500,
                  height=1800,
                  max_words=100,
                  colormap='tab10'
                  )

In [ ]:
fig, axes = plt.subplots(5, 4, figsize=(16, 25), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
  fig.add_subplot(ax)
  cloud.generate_from_frequencies(topic_words[i], max_font_size=300)
  plt.gca().imshow(cloud)
  plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
  plt.gca().axis('off')

plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()

## pyLDAvisという可視化ツールでトピックを可視化
* https://pyldavis.readthedocs.io/en/latest/

In [ ]:
!pip install pyLDAvis

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn

In [ ]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda, X, vectorizer, mds='tsne')
panel

## 05 gensimのLDAでトピック抽出

* gensimのLdaModelはデフォルトの設定だと正しく動かない
 * passesを20ぐらいにはしておくこと。
 * 下記Webページは使い方を間違っているので要注意（passesをデフォルト設定で使っている）
 http://www.ie110704.net/2018/12/29/wordcloud%E3%81%A8pyldavis%E3%81%AB%E3%82%88%E3%82%8Blda%E3%81%AE%E5%8F%AF%E8%A6%96%E5%8C%96%E3%81%AB%E3%81%A4%E3%81%84%E3%81%A6/
* gensimのperplexityはトークンあたりのELBOのnp.exp2()で求めている
 * 自然対数の底を使って求めたELBOをもとにして計算しているにもかかわらず。


In [ ]:
from gensim import corpora

In [ ]:
dictionary = corpora.Dictionary(new_docs)

In [ ]:
print(dictionary)

In [ ]:
dictionary.filter_extremes(no_below=10, no_above=0.5)

In [ ]:
len(dictionary)

In [ ]:
gs_corpus = [dictionary.doc2bow(doc) for doc in new_docs]

In [ ]:
print(gs_corpus[0])

In [ ]:
import logging

logging.basicConfig(filename='myapp.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
from gensim.models.ldamodel import LdaModel

In [ ]:
lda = LdaModel(corpus=gs_corpus, num_topics=n_components,
               passes=20)

In [ ]:
import numpy as np

np.exp(- lda.log_perplexity(gs_corpus))